In [1]:
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN

print("✅ PyTorch Version:", torch.__version__)
print("✅ CUDA Available:", torch.cuda.is_available())
print("✅ GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")
print("✅ NumPy Version:", np.__version__)
print("✅ OpenCV Version:", cv2.__version__)
print("✅ Facenet-PyTorch Loaded:", MTCNN is not None)


c:\Users\bchal\anaconda3\envs\mindcare-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ PyTorch Version: 2.6.0+cu126
✅ CUDA Available: True
✅ GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
✅ NumPy Version: 1.26.4
✅ OpenCV Version: 4.11.0
✅ Facenet-PyTorch Loaded: True


In [2]:
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN

# Initialize MTCNN for face detection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(keep_all=False, device=device)

# Open the camera
cap = cv2.VideoCapture(0)

# Get camera FPS (default to 30 if not available)
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0 or fps is None:
    fps = 30  # Default FPS

capture_time = 5  # Seconds of data needed for BPM estimation
frame_count = int(fps * capture_time)  # Convert to frames

print(f"🎥 Camera FPS: {fps}")
print("🟢 Keep your face steady. Calculating heart rate in real time...")

# Variables for heart rate estimation
frame_buffer = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Camera not detected. Exiting...")
        break

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect face
    boxes, _ = mtcnn.detect(rgb_frame)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Extract forehead ROI (upper part of the face)
            forehead_y1 = y1
            forehead_y2 = y1 + (y2 - y1) // 4  # Top 25% of face
            forehead_x1, forehead_x2 = x1, x2

            # Ensure ROI is within frame bounds
            if forehead_y2 > frame.shape[0] or forehead_x2 > frame.shape[1]:
                print("⚠️ Forehead region out of bounds")
                continue

            forehead_roi = frame[forehead_y1:forehead_y2, forehead_x1:forehead_x2]

            # Ensure ROI is not empty
            if forehead_roi.size == 0:
                print("⚠️ Empty forehead ROI, skipping frame...")
                continue

            # Convert to grayscale & extract mean intensity (for rPPG)
            forehead_gray = cv2.cvtColor(forehead_roi, cv2.COLOR_BGR2GRAY)
            mean_intensity = np.mean(forehead_gray)
            frame_buffer.append(mean_intensity)

            print(f"🔹 Collected Frames: {len(frame_buffer)} / {frame_count}")  # Debugging

            # Keep only the last N frames
            if len(frame_buffer) > frame_count:
                frame_buffer.pop(0)  # Remove oldest frame

                # Apply Fast Fourier Transform (FFT) for BPM estimation
                fft_result = np.fft.fft(frame_buffer)
                freqs = np.fft.fftfreq(len(fft_result), d=1/fps)

                # Filter valid frequency range (0.5 - 3.0 Hz for human heart rate)
                valid_indices = np.where((freqs > 0.5) & (freqs < 3.0))
                valid_freqs = freqs[valid_indices]
                valid_fft = np.abs(fft_result[valid_indices])

                if len(valid_freqs) > 0:
                    peak_freq = valid_freqs[np.argmax(valid_fft)]
                    bpm = abs(peak_freq * 60)  # Convert to beats per minute
                    print(f"✅ BPM Detected: {bpm:.2f}")  # Debugging output
                else:
                    bpm = 0  # No valid frequency detected
                    print("❌ No valid BPM detected.")  # Debugging output

                # Display BPM on frame
                cv2.putText(frame, f'BPM: {int(bpm)}', (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show real-time video
    cv2.imshow('Real-Time Heart Rate Detection', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\n🛑 Exiting program...")
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Camera released. All windows closed.")



🎥 Camera FPS: 30.0
🟢 Keep your face steady. Calculating heart rate in real time...
🔹 Collected Frames: 1 / 150
🔹 Collected Frames: 2 / 150
🔹 Collected Frames: 3 / 150
🔹 Collected Frames: 4 / 150
🔹 Collected Frames: 5 / 150
🔹 Collected Frames: 6 / 150
🔹 Collected Frames: 7 / 150
🔹 Collected Frames: 8 / 150
🔹 Collected Frames: 9 / 150
🔹 Collected Frames: 10 / 150
🔹 Collected Frames: 11 / 150
🔹 Collected Frames: 12 / 150
🔹 Collected Frames: 13 / 150
🔹 Collected Frames: 14 / 150
🔹 Collected Frames: 15 / 150
🔹 Collected Frames: 16 / 150
🔹 Collected Frames: 17 / 150
🔹 Collected Frames: 18 / 150
🔹 Collected Frames: 19 / 150
🔹 Collected Frames: 20 / 150
🔹 Collected Frames: 21 / 150
🔹 Collected Frames: 22 / 150
🔹 Collected Frames: 23 / 150
🔹 Collected Frames: 24 / 150
🔹 Collected Frames: 25 / 150
🔹 Collected Frames: 26 / 150
🔹 Collected Frames: 27 / 150
🔹 Collected Frames: 28 / 150
🔹 Collected Frames: 29 / 150
🔹 Collected Frames: 30 / 150
🔹 Collected Frames: 31 / 150
🔹 Collected Frames: 32 / 15